In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud ,STOPWORDS
from PIL import Image
import re
import gc
import sys
import time
import warnings
import json
import torch
import emoji
import random
import unicodedata
import multiprocessing
import seaborn as sns
from functools import partial, lru_cache
from tqdm import tqdm_notebook
from wordcloud import WordCloud, STOPWORDS
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
train = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv')
test = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv')
sub = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/sample_submission.csv')
import os
print(os.listdir("../input/nvidiaapex/repository/NVIDIA-apex-39e153a"))


In [ ]:
from sklearn import metrics
from sklearn import model_selection
import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
from nltk.stem import PorterStemmer
from sklearn.metrics import roc_auc_score
%load_ext autoreload
%autoreload 2
%matplotlib inline
from tqdm import tqdm, tqdm_notebook
import warnings
import pickle
from apex import amp
import shutil

In [ ]:
device=torch.device('cuda')

In [ ]:
MAX_SEQUENCE_LENGTH = 220
SEED = 23
EPOCHS = 1
DATA_DIR = "../input/jigsaw-unintended-bias-in-toxicity-classification"
INPUT_DIR = "../input"
WORK_DIR = "../working/"
LOAD = 1000000                         
VALIDATION = 100000  
TOXICITY_COLUMN = 'target'

In [ ]:
PACKAGE_DIR = "../input/ppbert/pytorch-pretrained-bert/pytorch-pretrained-BERT"
sys.path.append(PACKAGE_DIR)

from pytorch_pretrained_bert import convert_tf_checkpoint_to_pytorch
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification,BertAdam
from pytorch_pretrained_bert import BertConfig

In [ ]:
BERT_MODEL_PATH = '../input/bert-pretrained-models/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/'
convert_tf_checkpoint_to_pytorch.convert_tf_checkpoint_to_pytorch(
    BERT_MODEL_PATH + 'bert_model.ckpt',
BERT_MODEL_PATH + 'bert_config.json',
WORK_DIR + 'pytorch_model.bin')

shutil.copyfile(BERT_MODEL_PATH + 'bert_config.json', WORK_DIR + 'bert_config.json')

In [ ]:
bert_config = BertConfig('../input/bert-pretrained-models/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/'+'bert_config.json')

In [ ]:
def convert_lines(example):
    tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None,do_lower_case=True)
    MAX_SEQ_LENGTH=220
    MAX_SEQ_LENGTH -=2
    all_tokens = []
    longer = 0
    for text in example:
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>MAX_SEQ_LENGTH:
            tokens_a = tokens_a[:MAX_SEQ_LENGTH]
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (MAX_SEQ_LENGTH - len(tokens_a))
        all_tokens.append(one_token)
    #print(longer)
    return np.array(all_tokens)

In [ ]:
%%time
train = train.sample(LOAD+VALIDATION,random_state=SEED)
print('[INFO]Loaded %d records' % len(train))

train['comment_text'] = train['comment_text'].astype(str) 

with multiprocessing.Pool(processes=2) as pool:
     sequences = pool.map(convert_lines, train["comment_text"].fillna("NAN"))
        
#sequences = convert_lines(train["comment_text"].fillna("NAN"),MAX_SEQUENCE_LENGTH,tokenizer)
train = train.fillna(0)
# List all identities
identities = ['male','female','transgender','other_gender','heterosexual','homosexual_gay_or_lesbian',
              'bisexual','other_sexual_orientation','christian','jewish','muslim','hindu','buddhist',
              'atheist','other_religion','black','white','asian','latino','other_race_or_ethnicity',
              'physical_disability','intellectual_or_learning_disability','psychiatric_or_mental_illness',
              'other_disability']
races = ['black','white','asian','latino','other_race_or_ethnicity']
religions = ['atheist', 'buddhist', 'christian', 'hindu', 'muslim', 'jewish','other_religion']
sexual_orientation = ['heterosexual', 'homosexual_gay_or_lesbian', 'bisexual', 'other_sexual_orientation']

train = train.drop(['comment_text'],axis=1)
# convert target to 0,1
train['target']=(train['target']>=0.5).astype(float)

In [ ]:
X = sequences[:LOAD]                
y = train['target'].values[:LOAD]
X_val = sequences[LOAD:]                
y_val = train['target'].values[LOAD:]
test_df = train.tail(VALIDATION).copy()
train_df = train.head(LOAD)
train_dataset = torch.utils.data.TensorDataset(torch.tensor(X,dtype=torch.long), torch.tensor(y,dtype=torch.float))


In [ ]:
def init_model(): 
    lr=2e-5
    batch_size = 32
    accumulation_steps=2
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True
    model = BertForSequenceClassification.from_pretrained("../working",cache_dir=None,num_labels=1)
    model.zero_grad()
    model = model.to(device)
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]

    train = train_dataset

    num_train_optimization_steps = int(EPOCHS*len(train)/batch_size/accumulation_steps)

    optimizer = BertAdam(optimizer_grouped_parameters,
                         lr=lr,
                         warmup=0.05,
                         t_total=num_train_optimization_steps)

    model, optimizer = amp.initialize(model, optimizer, opt_level="O1",verbosity=0)

In [ ]:
OUPUT_MODEL_FILE = "bert_pytorch.bin"

init_model()

model=model.train()

tq = tqdm_notebook(range(EPOCHS))
for epoch in tq:
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    avg_loss = 0.
    avg_accuracy = 0.
    lossf=None
    tk0 = tqdm_notebook(enumerate(train_loader),total=len(train_loader),leave=False)
    optimizer.zero_grad()
    for i,(x_batch, y_batch) in tk0:
        y_pred = model(x_batch.to(device), attention_mask=(x_batch>0).to(device), labels=None)
        loss =  F.binary_cross_entropy_with_logits(y_pred,y_batch.to(device))
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
        if (i+1) % accumulation_steps == 0:             
            optimizer.step()
            optimizer.zero_grad()
        if lossf:
            lossf = 0.98*lossf+0.02*loss.item()
        else:
            lossf = loss.item()
        tk0.set_postfix(loss = lossf)
        avg_loss += loss.item() / len(train_loader)
        avg_accuracy += torch.mean(((torch.sigmoid(y_pred[:,0])>0.5) == (y_batch[:,0]>0.5).to(device)).to(torch.float) ).item()/len(train_loader)
    tq.set_postfix(avg_loss=avg_loss,avg_accuracy=avg_accuracy)


torch.save(model.state_dict(), OUTPUT_MODEL_FILE)

In [ ]:
model = BertForSequenceClassification(bert_config,num_labels=len(y_columns))
model.load_state_dict(torch.load(output_model_file ))
model.to(device)
for param in model.parameters():
    param.requires_grad=False
model.eval()
valid_preds = np.zeros((len(X_val)))
valid = torch.utils.data.TensorDataset(torch.tensor(X_val,dtype=torch.long))
valid_loader = torch.utils.data.DataLoader(valid, batch_size=32, shuffle=False)

tk0 = tqdm_notebook(valid_loader)
for i,(x_batch,)  in enumerate(tk0):
    pred = model(x_batch.to(device), attention_mask=(x_batch>0).to(device), labels=None)
    valid_preds[i*32:(i+1)*32]=pred[:,0].detach().cpu().squeeze().numpy()




In [ ]:
test_df['BERT']=torch.sigmoid(torch.tensor(valid_preds)).numpy()
test_df.to_csv("submission.csv",index=False)